In [1]:
import contextlib
import os

import numpy as np
import pandas as pd
from constants import max_steps, replicates

from mesa.batchrunner import batch_run_not_stupid
from mesa.examples.advanced.epstein_civil_violence.model import EpsteinCivilViolence

In [ ]:
files = [f"data_{x:04}.csv" for x in range(1, 3)]
files

['data_0003.csv']

In [3]:
def in_file(filename: str):
    return f"inputs/{filename}"


def out_file(filename: str):
    return f"outputs/{filename}"

In [4]:
def run_for_file(filename: str):
    data = pd.read_csv(in_file(filename), index_col=0)
    print(f"Loaded {len(data)} parameter samples.")

    result = batch_run_not_stupid(
        EpsteinCivilViolence,
        parameters=data,
        iterations=replicates,
        max_steps=max_steps,
        number_processes=None,
        display_progress=True,
    )

    ripley_police = np.zeros((len(result), len(result[0]["ripley_l_police"])))
    ripley_citizen = np.zeros((len(result), len(result[0]["ripley_l_citizen"])))
    for nr, res in enumerate(result):
        ripley_police[nr] = res["ripley_l_police"]
        del res["ripley_l_police"]
        ripley_citizen[nr] = res["ripley_l_citizen"]
        del res["ripley_l_citizen"]

    result_df = pd.DataFrame(result)
    with contextlib.suppress(FileExistsError):
        os.mkdir("outputs")
    result_df.to_csv(out_file(filename))
    np.save(out_file(f"ripley_police_{filename}"), ripley_police)
    np.save(out_file(f"ripley_citizen_{filename}"), ripley_citizen)

In [5]:
for file in files:
    run_for_file(file)

Loaded 5 parameter samples.


  0%|          | 0/500 [00:00<?, ?it/s]